<a href="https://colab.research.google.com/github/nimratmann/sqlite_database_operations/blob/main/sql_db_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import re

# Hospital 1: Stony Brook Hospital

In [ ]:
sb = pd.read_csv('https://raw.githubusercontent.com/nimratmann/sqlite_database_operations/main/Dataset/113243405_StonyBrookUniversityHospital_standardcharges.csv')
sb

## Data Exploration and Analysis

In [ ]:
print("Stony Brook Hospital Standard Charges Overview: ")
print(sb.head())

In [ ]:
print("Summary Statistics for Stony Brook Hospital Standard Charges: ")
print(sb.describe())

## Cleaning Data and Handling the Missing Values

In [ ]:
def clean_column_names(sb):
    # Create a dictionary to map old column names to cleaned names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()
    column_mapping = {col: clean_name(col) for col in sb.columns}

    # Use the rename method to rename columns
    sb = sb.rename(columns=column_mapping)

    return sb

# Apply the clean_column_names function to the DataFrame
sb = clean_column_names(sb)
sb

In [ ]:
print("Missing Values in Stony Brook Hospital Charges Dataset: ")
print(sb.isnull().sum())

## Basic Statistics

In [ ]:
print("Frequency Counts for the 'description column' in Stony Brook Hospital Charges dataset: ")
print(sb['description'].value_counts())

In [ ]:
# Descriptive Statitics for the Numerical Columns
numerical_columns = [
     "discountedcashprice",
    "grosscharge",
    "deidentifiedmincontractedrate",
    "deidentifiedmaxcontractedrate",
    "derivedcontractedrate",
    "1199commercialother",
    "aetnamedicareadvantagehmo",
    "aetnacommercialhmopos",
    "aetnacommercialppoopenaccess",
    "aetnacommercialother",
    "empirehealthcommercialother",
    "empirehealthcommercialppoopenaccess",
    "bluecrossblueshieldcommercialother",
    "beaconhealthcommercialother",
    "carelonhealthcommercialother",
    "cignacommercialppoopenaccess",
    "cignacommercialother",
    "cignacommercialhmopos",
    "ehfacetcommercialother",
    "emblemhealthcommercialppoopenaccess",
    "emblemhealthcommercialother",
    "emblemhealthcommercialhmopos",
    "emblemhealthmedicaidhmo",
    "emblemhealthmedicareadvantagehmo",
    "empirehealthcommercialhmopos",
     "empirehealthmedicareadvantagehmo",
    "empirehealthmedicaidhmo",
    "evernorthcommercialother",
    "fideliscommercialother",
    "fidelismedicareadvantagehmo",
    "fidelismedicaidhmo",
    "ghicommercialother",
    "healthfirstcommercialother",
    "healthfirstmedicareadvantagehmo",
    "healthfirstmedicaidhmo",
    "healthplushpmedicaidhmo",
    "healthplushpcommercialother",
    "healthplushpmedicareadvantagehmo",
    "humanacommercialother",
    "humanacommercialhmopos",
    "humanacommercialppoopenaccess",
    "meritainhealthcommercialother",
    "molinacommercialother",
    "optumcommercialother",
    "oxfordcommercialother",
    "oxfordcommercialhmopos",
    "tricarecommercialother",
    "unitedhealthcarecommercialother",
    "unitedhealthcaremedicareadvantagehmo",
    "unitedhealthcarecommercialhmopos",
    "unitedhealthcaremedicaidhmo",
    "unitedhealthcarecommercialppoopenaccess",
    "veteranfamilycommercialother",
]

analysis_results = {}
for column in numerical_columns:
    mean = sb[column].mean()
    median = sb[column].median()
    mode = sb[column].mode()
    std_dev = sb[column].std()
    min_value = sb[column].min()
    max_value = sb[column].max()

    # Calculate frequency counts distribution
    frequency_distribution = sb[column].value_counts().reset_index()
    frequency_distribution.columns = [column, "Frequency"]

    analysis_results[column] = {
        "Mean": mean,
        "Median": median,
        "Mode": mode,
        "Standard Deviation": std_dev,
        "Min Value": min_value,
        "Max Value": max_value,
        "Frequency Distribution": frequency_distribution
    }

# Display the analysis results
for column, results in analysis_results.items():
    print(f"Analysis for column: {column}")
    print(results)
    print("\n")


In [ ]:
sb_mean = sb['discountedcashprice'].mean()
sb_median = sb['discountedcashprice'].median()
sb_mode = sb['discountedcashprice'].mode().values[0]


sb_range = sb['discountedcashprice'].max() - sb['discountedcashprice'].min()
sb_var = sb['discountedcashprice'].var()
sb_std= sb['discountedcashprice'].std()
sb_iqr = sb['discountedcashprice'].quantile(0.75) - sb['discountedcashprice'].quantile(0.25)


print("Measures of Central Tendency:")
print(f"Mean: {sb_mean}")
print(f"Median: {sb_median}")
print(f"Mode: {sb_mode}")
print("\nMeasures of Spread:")
print(f"Range: {sb_range}")
print(f"Variance: {sb_var}")
print(f"Standard Deviation: {sb_std}")
print(f"IQR (Interquartile Range): {sb_iqr}")


In [ ]:
print (sb['description'].value_counts())
print (sb['type'].value_counts())
print (sb['code'].value_counts())
print (sb['packagelinelevel'].value_counts())

## Data Distribution

In [ ]:
plt.figure(figsize=(9,9))
plt.hist(sb['discountedcashprice'], bins=15, color='pink', edgecolor='black')
plt.title('The Frequency of Stony Brook Hospital Services Discounted Cash Price')
plt.xlabel('Discounted Cash Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(9,9))
plt.hist(sb['grosscharge'], bins=15, color='green', edgecolor='black')
plt.title('The Frequency of Stony Brook Hospital Services Gross Charge')
plt.xlabel('Gross Charge')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Hospital 2: NewYork-Presbyterian



In [ ]:
nyp = pd.read_json('NewYorkPresbyterianHospital_standardcharges.zip')
nyp

## Data Exploration and Analysis

In [ ]:
print("NewYork-Presbyterian Hospital Standard Charges Overview: ")
print(nyp.head())

In [ ]:
print("Summary Statistics for NewYork-Presbyterian Hospital Standard Charges: ")
print(nyp.describe())

## Cleaning Data and Handling the Missing Values

In [65]:
def clean_column_names(nyp):
    # Create a dictionary to map old column names to cleaned names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()
    column_mapping = {col: clean_name(col) for col in nyp.columns}

    # Use the rename method to rename columns
    nyp = nyp.rename(columns=column_mapping)

    return nyp

# Apply the clean_column_names function to the DataFrame
nyp = clean_column_names(nyp)
nyp

,codecptdrg,description,revcode,inpatientoutpatient,grosscharges,discountedcashprice,aetna,cigna,empirebluecrossblueshield,emblemhealth,...,consumerhealthnetwork,devon,equian,firsthealth,magnacare,multiplanbeechstreetphcs,qhm,worldwide,minimumnegotiatedcharge,maximumnegotiatedcharge
0,96360,HC IV INFUSION HYDRATION INITIAL 31 MIN-1HR,0260,Inpatient/Outpatient,866.000000,866.000000,641.706,494.486,640.84,1388.424,...,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,253.048250,1388.424000
1,96361,HC IV INFUSION HYDRATION FLUIDS ADDL HR,0260,Inpatient/Outpatient,358.000000,358.000000,265.278,204.418,264.92,1305.112,...,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,51.903250,1990.891341
2,96365,HC IV INFUSION FOR THER/PROPH/DIAG INITIAL UP ...,0260,Inpatient/Outpatient,1397.000000,1397.000000,1035.177,797.687,1033.78,1475.508,...,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,253.048250,1475.508000
3,96367,HC IV INF THER/PROPH/DIAG ADDL SEQ NEW DRUG UP...,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,82.650750,1325.120000
4,96368,HC IV INFUSION FOR THER/PROPH/DIAG CONCURRENT,0260,Inpatient/Outpatient,480.000000,480.000000,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,156.982800,1325.120000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,184095.129020,184095.129020,86723.5232,98911.694706,82299.84,115129.89148,...,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,83652.826627,12561.144027,115129.891480
6189,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Multiple,Inpatient,85385.941857,85385.941857,57748.8352,65864.888166,63931.455093,76664.51828,...,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,38799.37198,7833.746854,76664.518280
6190,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,411665.999955,411665.999955,114736.9984,130862.199072,214294.876425,152319.20576,...,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,187061.03038,21016.798830,214294.876425
6191,988,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Multiple,Inpatient,126522.871140,126522.871140,58654.7264,66898.093812,91907.6928,77867.13496,...,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,57491.992646,10429.428059,91907.692800


In [ ]:
print("Missing Values in NewYork-Presbyterian Hospital Charges Dataset: ")
print(nyp.isnull().sum())

## Basic Statistics

In [ ]:
print("Frequency Counts for the 'description column' in NewYork-Presbyterian Hospital Charges dataset: ")
print(nyp['description'].value_counts())

In [ ]:
print(nyp.columns)

In [ ]:
# Descriptive Statitics for the Numerical Columns
numerical_columns = [
    'grosscharges',
    'discountedcashprice',
    'minimumnegotiatedcharge',
    'maximumnegotiatedcharge',

]

analysis_results = {}
for column in numerical_columns:
    mean = nyp[column].mean()
    median = nyp[column].median()
    mode = nyp[column].mode().tolist()  # Convert mode to a list
    std_dev = nyp[column].std()
    min_value = nyp[column].min()
    max_value = nyp[column].max()

    # Calculate frequency counts distribution
    frequency_distribution = nyp[column].value_counts().reset_index()
    frequency_distribution.columns = [column, "Frequency"]

    analysis_results[column] = {
        "Mean": mean,
        "Median": median,
        "Mode": mode,
        "Standard Deviation": std_dev,
        "Min Value": min_value,
        "Max Value": max_value,
        "Frequency Distribution": frequency_distribution
    }

# Display the analysis results
for column, results in analysis_results.items():
    print(f"Analysis for column: {column}")
    print(results)
    print("\n")


In [ ]:
nyp_mean = nyp['discountedcashprice'].mean()
nyp_median = nyp['discountedcashprice'].median()
nyp_mode = nyp['discountedcashprice'].mode().values[0]


nyp_range = nyp['discountedcashprice'].max() - nyp['discountedcashprice'].min()
nyp_var = nyp['discountedcashprice'].var()
nyp_std= nyp['discountedcashprice'].std()
nyp_iqr = nyp['discountedcashprice'].quantile(0.75) - nyp['discountedcashprice'].quantile(0.25)


print("Measures of Central Tendency:")
print(f"Mean: {nyp_mean}")
print(f"Median: {nyp_median}")
print(f"Mode: {nyp_mode}")
print("\nMeasures of Spread:")
print(f"Range: {nyp_range}")
print(f"Variance: {nyp_var}")
print(f"Standard Deviation: {nyp_std}")
print(f"IQR (Interquartile Range): {nyp_iqr}")

In [ ]:
print (nyp['codecptdrg'].value_counts())
print (nyp['description'].value_counts())
print (nyp['revcode'].value_counts())
print (nyp['inpatientoutpatient'].value_counts())

## Data Distribution

In [ ]:
plt.figure(figsize=(9,9))
plt.hist(nyp['grosscharges'], bins=15, color='green', edgecolor='black')
plt.title('The Frequency of NewYork-Presbyterian Hospital Services Gross Charge')
plt.xlabel('Gross Charge')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(9,9))
plt.hist(nyp['discountedcashprice'], bins=15, color='pink', edgecolor='black')
plt.title('The Frequency of NewYork-Presbyterian Hospital Services Discounted Cash Price')
plt.xlabel('Discounted Cash Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

#  SQLite Database Operations

## Manual Table Creation


In [76]:
import sqlite3

# Connect to the SQLite database and create database
conn = sqlite3.connect('health.db')

cursor = conn.cursor()

    # Create the 'NewYork-Presbyterian' table if it doesn't exist
cursor.execute('''
        CREATE TABLE IF NOT EXISTS nyp
        (
            [codecptdrg] INTEGER PRIMARY KEY,
            [description] TEXT,
            [revcode] INTEGER,
            [inpatientoutpatient] TEXT,
            [grosscharges] REAL,
            [discountedcashprice] REAL
        )
    ''')

data_to_insert = [
        (1, 'Consultation', 123, 'Inpatient', 1000.50, 800.25),
        (2, 'Surgery', 456, 'Outpatient', 2500.75, 2000.60),
        (3, 'MRI Scan', 789, 'Inpatient', 150.20, 120.15)
    ]

    # Insert data into the 'NewYork-Presbyterian' table
    cursor.executemany('''
        INSERT INTO nyp
        VALUES (?, ?, ?, ?, ?, ?)
    ''', data_to_insert)

    # Commit the changes
    conn.commit()

    # Fetch and print the inserted data
    cursor.execute('SELECT * FROM nyp')
    tables = cursor.fetchall()

    for value in tables:
        print(value)

except sqlite3.Error as e:
    print("SQLite error:", e)
finally:
    conn.close()


IndentationError: ignored

In [ ]:
import sqlite3

    # Connect to the SQLite database and create database
conn = sqlite3.connect('health.db')
cursor = conn.cursor()

    # Create the 'NewYork-Presbyterian' table if it doesn't exist
cursor.execute('''
        CREATE TABLE IF NOT EXISTS nyp
        (
            [codecptdrg] INTEGER PRIMARY KEY,
            [description] TEXT,
            [revcode] INTEGER,
            [inpatientoutpatient] TEXT,
            [grosscharges] REAL,
            [discountedcashprice] REAL
        )
    ''')

data_to_insert = [
        (1, 'Consultation', 123, 'Inpatient', 1000.50, 800.25),
        (2, 'Surgery', 456, 'Outpatient', 2500.75, 2000.60),
        (3, 'MRI Scan', 789, 'Inpatient', 150.20, 120.15)
    ]

    # Insert data into the 'nyp' table
cursor.executemany('''
        INSERT INTO nyp
        VALUES (?, ?, ?, ?, ?, ?)
    ''', data_to_insert)

    # Commit the changes
conn.commit()

cursor.execute('''
    SELECT name
    FROM sqlite_master
    WHERE type= 'table'
''')

tables = cursor.fetchall()

for value in tables:
    print(value)



